In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas
import sys
sys.path.append('../')
from src.d06_reporting.Gentrification import Gentrification
from src.d04_modeling import propositional_classifier as pc
from src.d01_data.block_data_api import BlockDataApi
block_data_api = BlockDataApi()
block_df_dict = block_data_api.get_data(sfha=False).set_index('Block')
gentrification = Gentrification()

In [2]:
import splot
from splot.mapping import vba_choropleth

In [3]:
df = pd.read_csv("/share/data/school_choice_equity/displacement-typologies/data/outputs/typologies/SanFrancisco_typology_output.csv", sep=",")
frl_df_raw = block_data_api.get_data(frl=True)

In [4]:
gentrification.gentrification_data(df, block_df_dict)

KeyError: 'block database'

In [ ]:
ax = gentrification.frl_vs_ctip(frl_df_raw, block_df_dict)
plt.show()

In [ ]:
ax = gentrification.grouped_barchart(frl_df_raw, df)[0]
plt.show()

In [ ]:
gentrification.stacked_barchart(frl_df_raw, df)

In [ ]:
pc6 = pc.PropositionalClassifier([("pctAALPI", "pctFRL"), "pctBoth"], ["and", "or"])
pc6.statement

In [ ]:
pc7 = pc.PropositionalClassifier([("pctFocal", "pctBoth"), "BG_pctFocal", group_criterion="nbhd", ["or", "and"])

In [ ]:
params6 = [0.5, 0.8, 6] #parameters must match the features passed, in the order. Note the scale.
pred = pc6.get_solution_set(params6)
pred

In [ ]:
pc6_map = pc6.get_tiebreaker_map(params6, "geoid")
pc6_map.head(10)

In [ ]:
#pctFocal or pctBoth 
pc1 = pc.PropositionalClassifier(["pctAALPI", "pctFRL", "nBoth"], ["and", "or"])
pc1.statement

In [ ]:
grouped_Geoid = gentrification.gentrification_vs_demo(frl_df_raw, df)
grouped_Geoid.head(5)

In [ ]:
#gentrification_cols = ["OD", "ARG", "EOG", "AdvG", "SMMI", "ARE", "BE", "SAE"]
#gentrification_cols = [1 ,       2,     3,     4,     5,      6,     7,    8   ]
gentrification_values = []

for i in grouped_Geoid.index: 
    val = grouped_Geoid.loc[i, "Gentrification"]
    if val == "OD":
        gentrification_values.append(1)
    elif val == "ARG":
        gentrification_values.append(2)
    elif val == "EOG":
        gentrification_values.append(3)
    elif val == "AdvG":
        gentrification_values.append(4)
    elif val == "SMMI":
        gentrification_values.append(5)
    elif val == "ARE":
        gentrification_values.append(6)
    elif val == "BE":
        gentrification_values.append(7)
    elif val == "SAE":
        gentrification_values.append(8)

grouped_Geoid["New Gent"] = gentrification_values

In [ ]:
grouped_Geoid.head(10)

In [ ]:
pc6_map["New Geoid"] = pc6_map.index.astype(str).str[:10].astype(int)
pc6_map.head(5)

In [ ]:
merged_map = pc6_map.merge(grouped_Geoid, left_on = "New Geoid", right_index = True)
merged_map = geopandas.GeoDataFrame(merged_map)
merged_map.head(5)

In [ ]:
merged_map

In [ ]:
pc6.plot_map_new(merged_map, params6)

In [ ]:
solution_set = pc6.get_solution_set(params6)
solution_set

In [ ]:
merged_map.head(5)

In [ ]:
# counter = 1
# values = []
# for i in merged_map["New Geoid"]: 
#     if counter in solution_set: 
#         values.append(1)
#     else:
#         values.append(0)
#     counter += 1

In [ ]:
merged_map_filtered = merged_map[merged_map.index.isin(solution_set)]
merged_map_filtered

In [ ]:
plt.figure(figsize=((100,100)))
new = merged_map_filtered.plot(marker = 'o', color = 'green', zorder = 2)
new_map = pc6.plot_map_new(merged_map, params6, ax=new)
plt.show()